<a href="https://colab.research.google.com/github/Fatis092/repo52/blob/main/OCRFaris.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
!pip install opencv-python-headless pandas pytesseract

In [8]:
import cv2
import pytesseract
import pandas as pd
from google.colab.patches import cv2_imshow

In [9]:
pytesseract.pytesseract.tesseract_cmd = '/usr/bin/tesseract'

In [11]:
def detect_plate(frame):
   # Convert the frame to grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Apply edge detection
    edged = cv2.Canny(gray, 30, 200)

    # Find contours based on edges detected
    contours, _ = cv2.findContours(edged.copy(), cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

    # Sort contours by area
    contours = sorted(contours, key = cv2.contourArea, reverse = True)[:10]
    plate_contour = None

    # Loop over contours to find the rectangular plate area
    for contour in contours:

        # Approximate the contour
        epsilon = 0.018 * cv2.arcLength(contour, True)
        approx = cv2.approxPolyDP(contour, epsilon, True)

        # Check if the approximated contour has 4 points (rectangular)
        if len(approx) == 4:
            plate_contour = approx
            break

    if plate_contour is None:
        return None

    # Create a mask for the plate area and extract the plate
    x, y, w, h = cv2.boundingRect(plate_contour)
    plate = gray[y:y+h, x:x+w]

    return plate


In [12]:
# Function to extract text from the license plate
def extract_text_from_plate(plate_image):

    # Use Tesseract to perform OCR on the plate image
    plate_text = pytesseract.image_to_string(plate_image, lang='eng+ara')  # Support English and Arabic
    return plate_text.strip()

# Function to save plate data to CSV
def save_plate_to_csv(plate_text, csv_filename='plates.csv'):

    # Append to the CSV file only if plate_text is not empty
    if plate_text:
        df = pd.DataFrame({'Plate Number': [plate_text]})
        df.to_csv(csv_filename, mode='a', header=False, index=False)
        print(f"Saved plate {plate_text} to {csv_filename}")

# Step 3: Process video and detect plates
def process_video(video_path, csv_filename='plates.csv'):

    # Open the video file
    cap = cv2.VideoCapture(video_path)

    # Create a set to store unique plate numbers
    unique_plates = set()

    while cap.isOpened():

        ret, frame = cap.read()
        if not ret:
            break

        # Detect the plate in the current frame
        plate_image = detect_plate(frame)

        if plate_image is not None:

            # Extract text from the plate
            plate_text = extract_text_from_plate(plate_image)
            print(f"Detected Plate: {plate_text}")

            # If the plate text is new, save it to CSV
            if plate_text and plate_text not in unique_plates:
                unique_plates.add(plate_text)
                save_plate_to_csv(plate_text, csv_filename)

        # Optional: Display the current frame (for debugging)
        # cv2_imshow(frame)

    # Release the video capture object
    cap.release()

In [13]:
video_path = '/content/DJI_0064.mov'  # Path to your video
process_video(video_path, 'detected_plates.csv')

Detected Plate: 
Detected Plate: 
Detected Plate: 
Detected Plate: 
Detected Plate: 
Detected Plate: 
Detected Plate: 
Detected Plate: 
Detected Plate: 
Detected Plate: 
Detected Plate: 
Detected Plate: 
Detected Plate: 
Detected Plate: 
Detected Plate: 
Detected Plate: 
Detected Plate: 
Detected Plate: 
Detected Plate: 
Detected Plate: 
Detected Plate: 
Detected Plate: 
Detected Plate: 
Detected Plate: 
Detected Plate: 
Detected Plate: 
Detected Plate: 
Detected Plate: 
Detected Plate: 
Detected Plate: 
Detected Plate: 
Detected Plate: 
Detected Plate: 
Detected Plate: 
Detected Plate: 
Detected Plate: 
Detected Plate: 
Detected Plate: 
Detected Plate: 
Detected Plate: 
Detected Plate: 
Detected Plate: 
Detected Plate: 
Detected Plate: 
Detected Plate: 
Detected Plate: 
Detected Plate: 
Detected Plate: 
Detected Plate: 
Detected Plate: 
Detected Plate: 
Detected Plate: 
Detected Plate: 
Detected Plate: 
Detected Plate: 
Detected Plate: 
Detected Plate: 
Detected Plate: 
Detected Plate

In [16]:
from google.colab.patches import cv2_imshow

In [17]:
def process_video(video_path, csv_filename='plates.csv'):

    # Open the video file
    cap = cv2.VideoCapture(video_path)

    # Create a set to store unique plate numbers
    unique_plates = set()

    while cap.isOpened():

        ret, frame = cap.read()
        if not ret:
            break

        # Detect the plate in the current frame
        plate_image = detect_plate(frame)

        if plate_image is not None:

            # Extract text from the plate
            plate_text = extract_text_from_plate(plate_image)
            print(f"Detected Plate: {plate_text}")

            # If the plate text is new, save it to CSV
            if plate_text and plate_text not in unique_plates:
                unique_plates.add(plate_text)
                save_plate_to_csv(plate_text, csv_filename)

            # Display the frame with detected plate
            cv2_imshow(frame)

            # Add a delay to see the frame (e.g., 1 second)
            cv2.waitKey(1000)


    # Release the video capture object
    cap.release()
    cv2.destroyAllWindows()